# Wide & Deep Recommendation for large scale data - Deep Learning Model Training

This notebook demonstrates the distributed training for [Wide & Deep Learning](https://arxiv.org/abs/1606.07792) with the preprocessed features of the [Twitter Recsys Challenge 2021 dataset](http://www.recsyschallenge.com/2021/).

<img src="figures/overview-recsys.png" alt="overview-recsys" width="750"/>

First of all, we import the necessary packages in BigDL for cluster initialization and distributed training. Also import TensorFlow for Wide & Deep model definition.

In [2]:
import math
from time import time

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping

from bigdl.orca import init_orca_context, stop_orca_context, OrcaContext
from bigdl.orca.learn.tf2.estimator import Estimator
from bigdl.friesian.feature import FeatureTable

2022-03-23 10:55:39.569652: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-03-23 10:55:39.569679: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Initialize the environment on the YARN cluster. You simply need to prepare the Python environment on the driver node with [Anaconda](https://www.anaconda.com/products/individual) and BigDL will automatically distribute and prepare the environment for you across the cluster.
Besides, you can specify the allocated resources for this application during the initialization, including the number of nodes, cores and the amount of memory to use, etc. BigDL provides detailed guidance to be easily deployed on [Hadoop/YARN](https://bigdl.readthedocs.io/en/latest/doc/UserGuide/hadoop.html) or [K8S](https://bigdl.readthedocs.io/en/latest/doc/UserGuide/k8s.html) clusters.

In [4]:
# To display terminal's stdout and stderr in the Jupyter notebook.
OrcaContext.log_output = True

cluster_mode = "yarn"

executor_cores = 28
num_executor = 6
executor_memory = "30g"
driver_cores = 4
driver_memory = "36g"
conf = {"spark.executor.memoryOverhead": "130g",
        "spark.network.timeout": "10000000",
        "spark.sql.broadcastTimeout": "7200",
        "spark.sql.shuffle.partitions": "2000",
        "spark.locality.wait": "0s",
        "spark.sql.crossJoin.enabled": "true",
        "spark.task.cpus": "1",
        "spark.executor.heartbeatInterval": "200s",
        "spark.driver.maxResultSize": "40G",
        "spark.eventLog.enabled": "true",
        "spark.app.name": "recsys-demo-train"}

if cluster_mode == "local":  # For local machine
    sc = init_orca_context(cluster_mode="local",
                           cores=executor_cores, memory=executor_memory)
elif cluster_mode == "yarn":  # For Hadoop/YARN cluster
    sc = init_orca_context(cluster_mode="yarn", cores=executor_cores,
                           num_nodes=num_executor, memory=executor_memory,
                           driver_cores=driver_cores, driver_memory=driver_memory,
                           conf=conf, object_store_memory="80g",
                           env={"KMP_BLOCKTIME": "1",
                                "KMP_AFFINITY": "granularity=fine,compact,1,0",
                                "OMP_NUM_THREADS": "28"})


Initializing orca context
Current pyspark location is : /root/anaconda3/envs/bigdl/lib/python3.7/site-packages/pyspark/__init__.py
Initializing SparkContext for yarn-client mode
Start to pack current python env
Packing environment at '/root/anaconda3/envs/bigdl' to '/tmp/tmpmq670r4z/python_env.tar.gz'
[########################################] | 100% Completed | 13.4s
Packing has been completed: /tmp/tmpmq670r4z/python_env.tar.gz
pyspark_submit_args is: --master yarn --deploy-mode client --archives /tmp/tmpmq670r4z/python_env.tar.gz#python_env --driver-cores 4 --driver-memory 36g --num-executors 6 --executor-cores 28 --executor-memory 30g --driver-class-path /root/anaconda3/envs/bigdl/lib/python3.7/site-packages/bigdl/share/dllib/lib/bigdl-dllib-spark_2.4.6-2.0.0-jar-with-dependencies.jar:/root/anaconda3/envs/bigdl/lib/python3.7/site-packages/bigdl/share/orca/lib/bigdl-orca-spark_2.4.6-2.0.0-jar-with-dependencies.jar:/root/anaconda3/envs/bigdl/lib/python3.7/site-packages/bigdl/share/fr

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-03-23 10:55:59 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
2022-03-23 10:56:00 WARN  Client:66 - Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.
2022-03-23 10:57:08,470 Thread-5 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-03-23 10:57:08,471 Thread-5 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-03-23 10:57:08,471 Thread-5 WARN The bufferSize is set to 4000 but bufferedIo is false: false
2022-03-23 10:57:08,472 Thread-5 WARN The bufferSize is set to 4000 but bufferedIo is false: false
22-03-23 10:57:08 [Thread-5] INFO  Engine$:121 - Auto detect executor number and executor cores number
22-03-23 10:57:08 [Thread-5] INFO  Engine$:123 - Executor number is 6 and executor cores number is 28



User settings:

   KMP_AFFINITY=granularity=fine,compact,1,0
   KMP_BLOCKTIME=0
   KMP_SETTINGS=1
   OMP_NUM_THREADS=1

Effective settings:

   KMP_ABORT_DELAY=0
   KMP_ADAPTIVE_LOCK_PROPS='1,1024'
   KMP_ALIGN_ALLOC=64
   KMP_ALL_THREADPRIVATE=224
   KMP_ATOMIC_MODE=2
   KMP_BLOCKTIME=0
   KMP_CPUINFO_FILE: value is not defined
   KMP_DETERMINISTIC_REDUCTION=false
   KMP_DEVICE_THREAD_LIMIT=2147483647
   KMP_DISP_HAND_THREAD=false
   KMP_DISP_NUM_BUFFERS=7
   KMP_DUPLICATE_LIB_OK=false
   KMP_FORCE_REDUCTION: value is not defined
   KMP_FOREIGN_THREADS_THREADPRIVATE=true
   KMP_FORKJOIN_BARRIER='2,2'
   KMP_FORKJOIN_BARRIER_PATTERN='hyper,hyper'
   KMP_FORKJOIN_FRAMES=true
   KMP_FORKJOIN_FRAMES_MODE=3
   KMP_GTID_MODE=3
   KMP_HANDLE_SIGNALS=false
   KMP_HOT_TEAMS_MAX_LEVEL=1
   KMP_HOT_TEAMS_MODE=0
   KMP_INIT_AT_FORK=true
   KMP_ITT_PREPARE_DELAY=0
   KMP_LIBRARY=throughput
   KMP_LOCK_KIND=queuing
   KMP_MALLOC_POOL_INCR=1M
   KMP_MWAIT_HINTS=0
   KMP_NUM_LOCKS_IN_BLOCK=1
   KMP_

22-03-23 10:57:08 [Thread-5] INFO  ThreadPool$:95 - Set mkl threads to 1 on thread 28
2022-03-23 10:57:08 WARN  SparkContext:66 - Using an existing SparkContext; some configuration may not take effect.
22-03-23 10:57:08 [Thread-5] INFO  Engine$:446 - Find existing spark context. Checking the spark conf...
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.Sample
BigDLBasePickler registering: bigdl.dllib.utils.common  Sample
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.EvaluatedResult
BigDLBasePickler registering: bigdl.dllib.utils.common  EvaluatedResult
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.JTensor
BigDLBasePickler registering: bigdl.dllib.utils.common  JTensor
cls.getname: com.intel.analytics.bigdl.dllib.utils.python.api.JActivity
BigDLBasePickler registering: bigdl.dllib.utils.common  JActivity


Load the preprocessed train and validation data.

In [5]:
train_tbl = FeatureTable.read_parquet("/path/to/preprocessed/train/data")
valid_tbl = FeatureTable.read_parquet("/path/to/preprocessed/valid/data")

Define different features for Wide & Deep inputs:

- wide_cols and cross_cols would be one hot encoded to be fed into the Linear layer.
- embedding_cols would be converted to embedding vectors to be fed into the DNN.
- indicator_cols would be one hot encoded to be fed into the DNN.
- continuous_cols would be directly fed into the DNN.

In [8]:
wide_cols = ['engaged_with_user_is_verified', 'enaging_user_is_verified']
wide_dims = [1, 1]
cross_cols = ['present_media_language']
cross_dims = [600]

embedding_cols = []
embedding_dims = []

cat_cols = ['present_media',
            'tweet_type',
            'language']
cat_dims = [12, 2, 66]
count_cols = ['engaged_with_user_follower_count',
              'engaged_with_user_following_count',
              'enaging_user_follower_count',
              'enaging_user_following_count']
count_dims = [7, 7, 7, 7]
indicator_cols = cat_cols + count_cols
indicator_dims = cat_dims + count_dims

continuous_cols = ['len_hashtags',
                   'len_domains',
                   'len_links']

column_info = { "wide_base_cols": wide_cols,
                "wide_base_dims": wide_dims,
                "wide_cross_cols": cross_cols,
                "wide_cross_dims": cross_dims,
                "indicator_cols": indicator_cols,
                "indicator_dims": indicator_dims,
                "continuous_cols": continuous_cols,
                "embed_cols": [],
                "embed_in_dims": [],
                "embed_out_dims": [],
                "label": "label"}

Define the Wide & Deep model with TensorFlow Keras API.

In [9]:
def build_model(column_info, hidden_units=[100, 50, 25]):
    """Build an estimator appropriate for the given model type."""
    wide_base_input_layers = []
    wide_base_layers = []
    for i in range(len(column_info["wide_base_cols"])):
        wide_base_input_layers.append(tf.keras.layers.Input(shape=[], dtype="int32"))
        wide_base_layers.append(tf.keras.backend.one_hot(wide_base_input_layers[i], column_info["wide_base_dims"][i] + 1))

    wide_cross_input_layers = []
    wide_cross_layers = []
    for i in range(len(column_info["wide_cross_cols"])):
        wide_cross_input_layers.append(tf.keras.layers.Input(shape=[], dtype="int32"))
        wide_cross_layers.append(tf.keras.backend.one_hot(wide_cross_input_layers[i], column_info["wide_cross_dims"][i]))

    indicator_input_layers = []
    indicator_layers = []
    for i in range(len(column_info["indicator_cols"])):
        indicator_input_layers.append(tf.keras.layers.Input(shape=[], dtype="int32"))
        indicator_layers.append(tf.keras.backend.one_hot(indicator_input_layers[i], column_info["indicator_dims"][i] + 1))

    embed_input_layers = []
    embed_layers = []
    for i in range(len(column_info["embed_in_dims"])):
        embed_input_layers.append(tf.keras.layers.Input(shape=[], dtype="int32"))
        iembed = tf.keras.layers.Embedding(column_info["embed_in_dims"][i] + 1,
                                           output_dim=column_info["embed_out_dims"][i])(embed_input_layers[i])
        flat_embed = tf.keras.layers.Flatten()(iembed)
        embed_layers.append(flat_embed)

    continuous_input_layers = []
    continuous_layers = []
    for i in range(len(column_info["continuous_cols"])):
        continuous_input_layers.append(tf.keras.layers.Input(shape=[]))
        continuous_layers.append(tf.keras.layers.Reshape(target_shape=(1,))(continuous_input_layers[i]))

    if len(wide_base_layers + wide_cross_layers) > 1:
        wide_input = tf.keras.layers.concatenate(wide_base_layers + wide_cross_layers, axis=1)
    else:
        wide_input = (wide_base_layers + wide_cross_layers)[0]
    wide_out = tf.keras.layers.Dense(1)(wide_input)
    if len(indicator_layers + embed_layers + continuous_layers) > 1:
        deep_concat = tf.keras.layers.concatenate(indicator_layers +
                                                  embed_layers +
                                                  continuous_layers, axis=1)
    else:
        deep_concat = (indicator_layers + embed_layers + continuous_layers)[0]
    linear = deep_concat
    for ilayer in range(0, len(hidden_units)):
        linear_mid = tf.keras.layers.Dense(hidden_units[ilayer])(linear)
        bn = tf.keras.layers.BatchNormalization()(linear_mid)
        relu = tf.keras.layers.ReLU()(bn)
        dropout = tf.keras.layers.Dropout(0.1)(relu)
        linear = dropout
    deep_out = tf.keras.layers.Dense(1)(linear)
    added = tf.keras.layers.add([wide_out, deep_out])
    out = tf.keras.layers.Activation("sigmoid")(added)
    model = tf.keras.models.Model(wide_base_input_layers +
                                  wide_cross_input_layers +
                                  indicator_input_layers +
                                  embed_input_layers +
                                  continuous_input_layers,
                                  out)

    return model

The following figure shows the architecture of the Wide & Deep model.

<img src="figures/wnd.png" alt="wnd" width="750"/>

Set the hyperparameters for model training.

In [11]:
config = {
    "lr": 0.0001,
    "column_info": column_info,
    "inter_op_parallelism": 4,
    "intra_op_parallelism": 24
}
batch_size = 25600

Define the model_creator to create the Wide & Deep model on each node and compile it with optimizer, loss and metrics.

In [12]:
def model_creator(config):
    model = build_model(column_info=config["column_info"],
                        hidden_units=[1024, 1024])
    optimizer = tf.keras.optimizers.Adam(config["lr"])
    model.compile(optimizer=optimizer,
                  loss='binary_crossentropy',
                  metrics=['binary_accuracy', 'binary_crossentropy', 'AUC', 'Precision', 'Recall'])
    return model

The training architecture is shown in the following diagram. Our distributed training is implemented based on [RayOnSpark](https://medium.com/riselab/rayonspark-running-emerging-ai-applications-on-big-data-clusters-with-ray-and-analytics-zoo-923e0136ed6a) and [RaySGD](https://medium.com/distributed-computing-with-ray/faster-and-cheaper-pytorch-with-raysgd-a5a44d4fd220). On the underlying YARN cluster, the Spark Driver first launches multiple Spark executors across the cluster and the existing SparkContext creates a RayContext to launch Ray in the same cluster. The Ray processes stay alongside Spark executors and are managed by Spark executors. TensorFlow runners exist in Ray processes and they would take the local Spark in-memory data partitions for model training. TensorFlow runners communicate and synchronize with each other via [TensorFlow distribute MirroredStrategy](https://www.tensorflow.org/api_docs/python/tf/distribute/MirroredStrategy).

The cluster setup and implementation details are transparent to users. Users simply need to create a TensorFlow Estimator in BigDL for the entire process.

<img src="figures/e2etrain.png" alt="e2etrain" width="800"/>

Create the TensorFlow Estimator in BigDL for distributed Wide & Deep training.

In [14]:
estimator = Estimator.from_keras(
    model_creator=model_creator,
    verbose=True,
    config=config)

Launching Ray on cluster with Spark barrier mode


Start to launch ray driver on local
Executing command: ray start --address 172.168.0.107:38853 --redis-password 123456 --num-cpus 0 --object-store-memory 80000000000 --node-ip-address 172.168.0.101
2022-03-23 10:58:43,531	INFO scripts.py:747 -- Local node IP: 172.168.0.101
2022-03-23 10:58:43,654	SUCC scripts.py:755 -- --------------------
2022-03-23 10:58:43,654	SUCC scripts.py:756 -- Ray runtime started.
2022-03-23 10:58:43,654	SUCC scripts.py:757 -- --------------------
2022-03-23 10:58:43,654	INFO scripts.py:759 -- To terminate the Ray runtime, run
2022-03-23 10:58:43,655	INFO scripts.py:760 --   ray stop

E0323 10:58:43.621758087  303499 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
E0323 10:58:43.629680754  303499 fork_posix.cc:70]           Fork support is only compatible with the epoll1 and poll polling strategies
[2022-03-23 10:58:43,653 I 303499 303499] global_state_accessor.cc:360: This node has an IP address of 172.1

2022-03-23 10:58:44,960	INFO worker.py:843 -- Connecting to existing Ray cluster at address: 172.168.0.107:38853


{'node_ip_address': '172.168.0.101', 'raylet_ip_address': '172.168.0.101', 'redis_address': '172.168.0.107:38853', 'object_store_address': '/tmp/ray/session_2022-03-23_10-58-37_224781_259004/sockets/plasma_store', 'raylet_socket_name': '/tmp/ray/session_2022-03-23_10-58-37_224781_259004/sockets/raylet', 'webui_url': '172.168.0.107:8265', 'session_dir': '/tmp/ray/session_2022-03-23_10-58-37_224781_259004', 'metrics_export_port': 56881, 'node_id': '21217676fa1a3e4eee9e05be637197366ff94ddb13ed6ecc56433c9b'}


(Worker pid=259340, ip=172.168.0.107) 2022-03-23 10:58:46.251583: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: python_env/lib:python_env/lib/python3.7/lib-dynload::/opt/cloudera/parcels/CDH-5.15.2-1.cdh5.15.2.p0.3/lib/hadoop/lib/native
(Worker pid=259340, ip=172.168.0.107) 2022-03-23 10:58:46.251618: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
(Worker pid=279949, ip=172.168.0.102) 2022-03-23 10:58:46.293903: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: python_env/lib:python_env/lib/python3.7/lib-dynload::/opt/cloudera/parcels/CDH-5.15.2-1.cdh5.15.2.p0.3/lib/

We evaluate on the validation dataset every 50 iterations. We add an EarlyStopping callback so that when validation auc keeps dropping for three rounds, the training would be stopped.

In [15]:
train_count = train_tbl.size()
print("Total number of train records: {}".format(train_count))
total_steps = math.ceil(train_count / batch_size)
steps_per_epoch = 50
epochs = math.ceil(total_steps / steps_per_epoch)   # To train the full dataset for an entire epoch
val_count = valid_tbl.size()
print("Total number of val records: {}".format(val_count))
val_steps = math.ceil(val_count / batch_size)

callbacks = [EarlyStopping(monitor='val_auc', mode='max',verbose=1, patience=3)]

2022-03-23 10:59:01 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


Total number of train records: 747694282
Total number of val records: 14461760


Great! Let's launch the distributed training across the underlying YARN cluster! The training progress and the metrics at each step would be printed out during the training.

In [18]:
def label_cols(column_info):
    return [column_info["label"]]

def feature_cols(column_info):
    return column_info["wide_base_cols"] + column_info["wide_cross_cols"] +\
                  column_info["indicator_cols"] + column_info["embed_cols"] + column_info["continuous_cols"]

start = time()
estimator.fit(data=train_tbl.df,
              epochs=epochs,
              batch_size=batch_size,
              steps_per_epoch=steps_per_epoch,
              validation_data=valid_tbl.df,
              validation_steps=val_steps,
              callbacks=callbacks,
              feature_cols=feature_cols(column_info),
              label_cols=label_cols(column_info))
end = time()
print("Training time is: ", end - start)

Try to unpersist an uncached rdd


(Worker pid=314929, ip=172.168.0.106) Epoch 1/585


(Worker pid=314929, ip=172.168.0.106) 2022-03-23 11:13:27.299909: W tensorflow/core/framework/dataset.cc:768] Input of GeneratorDatasetOp::Dataset will not be optimized because the dataset does not implement the AsGraphDefInternal() method needed to apply optimizations.


48/50 [===========================>..] - ETA: 0s - loss: 0.6910 - binary_accuracy: 0.5943 - binary_crossentropy: 0.6910 - auc: 0.6279 - precision: 0.5854 - recall: 0.6343


50/50 [==============================] - 45s 815ms/step - loss: 0.6898 - binary_accuracy: 0.5950 - binary_crossentropy: 0.6898 - auc: 0.6289 - precision: 0.5862 - recall: 0.6340 - val_loss: 0.6879 - val_binary_accuracy: 0.5504 - val_binary_crossentropy: 0.6879 - val_auc: 0.5803 - val_precision: 0.5845 - val_recall: 0.3878
(Worker pid=314929, ip=172.168.0.106) Epoch 2/585
45/50 [==========================>...] - ETA: 0s - loss: 0.6599 - binary_accuracy: 0.6154 - binary_crossentropy: 0.6599 - auc: 0.6575 - precision: 0.6108 - recall: 0.6256


...

50/50 [==============================] - 39s 782ms/step - loss: 0.6429 - binary_accuracy: 0.6338 - binary_crossentropy: 0.6429 - auc: 0.6803 - precision: 0.6291 - recall: 0.6451 - val_loss: 0.6274 - val_binary_accuracy: 0.6565 - val_binary_crossentropy: 0.6274 - val_auc: 0.7044 - val_precision: 0.6574 - val_recall: 0.6719
(Worker pid=314929, ip=172.168.0.106) Epoch 18/585
46/50 [==========================>...] - ETA: 0s - loss: 0.6423 - binary_accuracy: 0.6346 - binary_crossentropy: 0.6423 - auc: 0.6815 - precision: 0.6293 - recall: 0.6447


50/50 [==============================] - ETA: 0s - loss: 0.6423 - binary_accuracy: 0.6346 - binary_crossentropy: 0.6423 - auc: 0.6815 - precision: 0.6293 - recall: 0.6444


50/50 [==============================] - 40s 806ms/step - loss: 0.6423 - binary_accuracy: 0.6346 - binary_crossentropy: 0.6423 - auc: 0.6815 - precision: 0.6293 - recall: 0.6444 - val_loss: 0.6273 - val_binary_accuracy: 0.6566 - val_binary_crossentropy: 0.6273 - val_auc: 0.7046 - val_precision: 0.6588 - val_recall: 0.6676
(Worker pid=314929, ip=172.168.0.106) Epoch 19/585
47/50 [===========================>..] - ETA: 0s - loss: 0.6422 - binary_accuracy: 0.6346 - binary_crossentropy: 0.6422 - auc: 0.6812 - precision: 0.6294 - recall: 0.6443


50/50 [==============================] - ETA: 0s - loss: 0.6421 - binary_accuracy: 0.6346 - binary_crossentropy: 0.6421 - auc: 0.6813 - precision: 0.6294 - recall: 0.6445


(Worker pid=226680, ip=172.168.0.103) Epoch 19: early stopping
(Worker pid=259340, ip=172.168.0.107) Epoch 19: early stopping
(Worker pid=314019, ip=172.168.0.104) Epoch 19: early stopping
(Worker pid=310081, ip=172.168.0.105) Epoch 19: early stopping
(Worker pid=279949, ip=172.168.0.102) Epoch 19: early stopping
(Worker pid=314929, ip=172.168.0.106) 
50/50 [==============================] - 39s 796ms/step - loss: 0.6421 - binary_accuracy: 0.6346 - binary_crossentropy: 0.6421 - auc: 0.6813 - precision: 0.6294 - recall: 0.6445 - val_loss: 0.6272 - val_binary_accuracy: 0.6568 - val_binary_crossentropy: 0.6272 - val_auc: 0.7044 - val_precision: 0.6566 - val_recall: 0.6757
(Worker pid=314929, ip=172.168.0.106) Epoch 19: early stopping
Training time is:  1185.4728910923004


After 19 rounds of training, we get validation [auc](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) 0.7044, [precision](https://en.wikipedia.org/wiki/Precision_and_recall) 0.6566 and [recall](https://en.wikipedia.org/wiki/Precision_and_recall) 0.6757. The total training time is around half an hour.

After the training, we save the trained model to be used for serving.

In [19]:
model = estimator.get_model()
tf.saved_model.save(model, "recsys_wnd/")

stop_orca_context()

2022-03-23 11:26:01.562497: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-03-23 11:26:01.562564: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-03-23 11:26:01.562601: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aep-001): /proc/driver/nvidia/version does not exist
2022-03-23 11:26:01.563329: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-23 11:26:02.425917: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequenc

INFO:tensorflow:Assets written to: recsys_wnd/assets
Stopping orca context
